<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part One: Data Preparation </h2>	


<h3>  Import Statements </h3>	


In [1]:
#Import anything you need here
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import skimage
from skimage import data
from skimage import io
import os
import time
import ray
ray.init(include_webui=False, num_cpus=4)

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:62788 to respond...
Waiting for redis server at 127.0.0.1:27851 to respond...
Starting the Plasma object store with 6.00 GB memory.
Starting local scheduler with the following resources: {'CPU': 4, 'GPU': 0}.


{'local_scheduler_socket_names': ['/tmp/scheduler74387804'],
 'node_ip_address': '10.0.0.56',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store58322992', manager_name='/tmp/plasma_manager67488520', manager_port=24080)],
 'raylet_socket_names': [],
 'redis_address': '10.0.0.56:62788',
 'webui_url': ''}

<h4> Using the code for reference, take the given folder and create a dataframe with the picture object, and the encoding as listed below. </h4>	
0=Airplanes, 1=Bear, 2=Blimp, 3=Comet, 4=Crab, 5=Dog, 6=Dolphin, 7=Giraffe, 8=Goat, 9=Gorilla, 10=Kangaroo, 11=Killer-Whale, 12=Leopards, 13=Llama, 14= Penguin, 15= Porcupine, 16=Teddy-Bear, 17=Triceratops, 18=Unicorn, 19=Zebra

In [2]:
def get_transformed_image_from_path(file_path, outFile_size):
    transformed_image = skimage.transform.resize(io.imread(file_path), outFile_size)
    return transformed_image

@ray.remote
def get_transformed_image_from_path_remote(file_path, outFile_size):
    return get_transformed_image_from_path(file_path, outFile_size)

def read_organize_data(dir_path, outFile_size):
    #Stores encodings
    
    category_list = ["Airplanes","Bear","Blimp","Comet","Crab","Dog","Dolphin","Giraffe",
                     "Goat","Gorilla","Kangaroo","Killer-Whale","Leopards","Llama",
                     "Penguin","Porcupine","Teddy-Bear","Triceratops","Unicorn","Zebra"]
    category_list = list(map(lambda x: x.lower(), category_list))
    category_encodings = dict(zip(category_list, range(len(category_list))))
# #     Uncomment to view category encoding dict
#     print(category_encodings)
    
    image_df = pd.DataFrame(columns = ['Pictures', 'Encoding'])
    data, encodings = [], []
    for category in os.listdir(dir_path):
        if category[0] != '.':    #Protecting against temporary files (Windows fix)
            for file_name in os.listdir(dir_path + category):
                if file_name[0] != '.':
                    file_encode = category_encodings[category]
                    file_path = dir_path + category + '/' + file_name
                    image_data = get_transformed_image_from_path_remote.remote(file_path, outFile_size) #
# #                     Uncomment to view thumbnails for all training images
#                     plt.figure()
#                     io.imshow(image_data)

                    encodings.append(file_encode)
                    data.append(image_data)
    
    image_df['Encoding'] = encodings        
    image_df['Pictures'] = ray.get(data) 
    return image_df

In [3]:
data_dir = "data/20_categories_training/"
resize_to = (128, 128)
start_time = time.time()

starting_data = read_organize_data(data_dir, resize_to)
end_time = time.time()
print("Time to compute results: {} seconds".format(end_time - start_time))



# starting_data

Time to compute results: 17.147059202194214 seconds


In [4]:
display(sum(starting_data["Encoding"].value_counts()))

1501